# Quick Howto get all comments from posts found with pushshift
How to search for all comments containing a word, get all comments from the post containing that comment, and store in a dataframe.
  
I prefer [PSAW](https://github.com/dmarx/psaw) over [PMAW](https://github.com/mattpodolak/pmaw) because it returns [PRAW](https://praw.readthedocs.io/en/stable/) objects and I am irked how PMAW sets up it's own logger. 

In [1]:
from secret_services import reddit,psaw_pushshift
import utils
import tqdm

In [2]:
import pandas as pd

#pmaw example - returns json
subreddit_name="environment"
word_to_check="companies"

In [3]:
#psaw example - returns praw objects
subreddit_name="environment"
word_to_check="companies"
comments=psaw_pushshift.search_comments(q=word_to_check, subreddit=subreddit_name, limit=200, before=1629990795)
import pandas as pd

post_with_comments=[]
for comment in comments:
    if word_to_check in comment.body.lower():#case insensitive check
        post_with_comments.append(
            {"comment_id": comment.id, "comment_text": comment.body,"score": comment.score,"post": comment.submission.id
            }
        )
    else:
        #edited or removed comments don't work
        print(comment.body)
df=pd.DataFrame(post_with_comments)
df


C:\Users\minim\miniconda3\envs\subreddit_analysis\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


[deleted]
[deleted]
[deleted]
[removed]
[deleted]
[removed]
[deleted]
[deleted]
[deleted]
[deleted]
[deleted]
[deleted]
[deleted]


,comment_id,comment_text,score,post
0,hafl3n0,I don't artificially inseminate..... my cows f...,-1,p9wdlq
1,haf6krp,Why the fuck cut down the last of the old grow...,15,pbm2jq
2,haemz66,I was beginning to believe that the vegan move...,1,pbbq9k
3,hac5sym,Its just a move by oil companies to keep doing...,7,pbhpam
4,habovkv,"There's going to be a fight, but it is looking...",2,pbh9tg
...,...,...,...,...
182,h8h73ow,You’re being hyperbolic. Vladimir Lenin and Fi...,1,p18k12
183,h8go124,"To a certain extent, oil companies merely prov...",1,p1vpy8
184,h8gkru2,I highly doubt that goes through. So many Demo...,5,p1ts7y
185,h8gh77h,"first off, we owe the overwhelming majority of...",1,p1v3b1


In [19]:
reddit.submission(df['post'].iloc[5]).url

'https://v.redd.it/r8nakgmcraj71'

In [7]:
full_discussion_rows=[]
for post_id in tqdm.tqdm(df['post'].iloc[:5]):#just doing first 5, traverse posts is super slow with 6th post ID and takes a while to do it https://www.reddit.com/r/environment/comments/pam1cx/the_colorado_river_that_supplies_water_to_40/
    comments=utils.traverse_post(reddit.submission(post_id))
    for comment,level in comments:
        full_discussion_rows.append({"comment_id": comment.id, "comment_text": comment.body,"score": comment.score,"post": post_id, 
     "level": level
            })
full_discussion_df=pd.DataFrame(full_discussion_rows)
full_discussion_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.24it/s]


,comment_id,comment_text,score,post,level
0,ha15f6s,"Yes! Tax plastics, tax meat production, etc. I...",237,p9wdlq,1
1,ha1du14,Plastics is much harder to accurately tax than...,45,p9wdlq,2
2,ha22vw7,Just start banning single use plastics anytime...,15,p9wdlq,3
3,ha2bpqi,"We do it with redemption values, those are an ...",12,p9wdlq,3
4,ha2f04u,"Oh, definitely, it's just that hose aspulls ca...",10,p9wdlq,4
...,...,...,...,...,...
299,habovkv,"There's going to be a fight, but it is looking...",2,pbh9tg,1
300,habqpnj,Alaska Election Info\n\n[Register to Vote](htt...,2,pbh9tg,1
301,habqqui,"Would you look at that, all of the words in yo...",1,pbh9tg,2
302,hac33vt,How many bots are there on Reddit anyway?,1,pbh9tg,3


In [16]:
full_discussion_df.to_csv("all_comments_from_found_posts_with_pushshift.csv",index=False)